# MLB Betting Model Data Collection

In [1]:
import pybaseball as pb
import pandas as pd
import numpy as np
import requests
import json
import odds

## Importing data from fangraphs using pybaseball api

In [2]:
def game_logs(season):
    teams = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'MIA', 'HOU', 'KCR', 'LAA', 'LAD', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SFG', 'SEA', 'STL', 'TBR', 'TEX', 'TOR', 'WSN'] #List of teams
    data_list = []   # Create an empty list for appending our dataframes to
    for team in teams:
        batting_logs = pb.team_game_logs(season, team)    # Use pybaseball API to pull game logs for each team
        df_temp = batting_logs[['Date', 'Home', 'Opp', 'R', 'H', 'HR', 'BA', 'OPS', 'OppStart']].copy()    # Only need certain columns
        df_temp['Team'] = team   # Add team name column
        data_list.append(df_temp)  # Append to list
    season_logs = pd.concat(data_list, ignore_index = True)  # Concatenate all teams dataframes
    print('Nulls:')
    print(season_logs.isnull().sum())
    print('='*100)
    print(season_logs.head())
    return season_logs

In [3]:
df = game_logs(2022)  # Get games from 2022 season

Nulls:
Date        0
Home        0
Opp         0
R           0
H           0
HR          0
BA          0
OPS         0
OppStart    0
Team        0
dtype: int64
     Date   Home  Opp  R  H  HR     BA    OPS        OppStart Team
0   Apr 7  False  SDP  4  3   1  0.115  0.554   Y.Darvish(71)  ARI
1   Apr 8  False  SDP  0  2   0  0.091  0.364    S.Manaea(83)  ARI
2   Apr 9  False  SDP  2  5   1  0.116  0.426  J.Musgrove(62)  ARI
3  Apr 10  False  SDP  5  5   2  0.129  0.516  N.Crismatt(58)  ARI
4  Apr 12  False  HOU  1  4   1  0.130  0.518    L.Garcia(57)  ARI


In [4]:
pitching = pb.pitching_stats(2022, qual = 0) # Pitching stats for players 2022

In [5]:
pitching.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
192,16137,2022,Carlos Rodon,SFG,29,13,8,5.7,2.84,29,...,18.7,24,0.060,112.5,159,0.399,398,0.163,0.304,2.69
291,16149,2022,Aaron Nola,PHI,29,9,12,5.5,3.38,29,...,12.6,36,0.073,112.0,157,0.319,492,0.196,0.318,2.79
300,14107,2022,Kevin Gausman,TOR,31,12,10,5.2,3.45,28,...,12.4,38,0.084,116.6,174,0.387,450,0.141,0.295,3.42
88,8700,2022,Justin Verlander,HOU,39,17,3,5.1,1.78,25,...,16.6,28,0.069,110.9,141,0.349,404,0.147,0.264,2.69
129,18684,2022,Sandy Alcantara,MIA,26,13,8,5.1,2.37,30,...,5.6,32,0.055,114.8,231,0.396,584,0.153,0.270,3.02


In [6]:
pitching.columns.values

array(['IDfg', 'Season', 'Name', 'Team', 'Age', 'W', 'L', 'WAR', 'ERA',
       'G', 'GS', 'CG', 'ShO', 'SV', 'BS', 'IP', 'TBF', 'H', 'R', 'ER',
       'HR', 'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB', 'FB', 'LD',
       'IFFB', 'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU', 'BUH',
       'K/9', 'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP',
       'LOB%', 'FIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB',
       'IFH%', 'BUH%', 'Starting', 'Start-IP', 'Relieving', 'Relief-IP',
       'RAR', 'Dollars', 'tERA', 'xFIP', 'WPA', '-WPA', '+WPA', 'RE24',
       'REW', 'pLI', 'inLI', 'gmLI', 'exLI', 'Pulls', 'WPA/LI', 'Clutch',
       'FB% 2', 'FBv', 'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%',
       'CHv', 'SF%', 'SFv', 'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL',
       'wCT', 'wCB', 'wCH', 'wSF', 'wKN', 'wFB/C', 'wSL/C', 'wCT/C',
       'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%',
       'F-Str

In [7]:
pitchers = pitching[['Name', 'ERA', 'K/BB', 'HR/9', 'WHIP', 'xFIP', 'HardHit%']].copy()   # Only need certain columns

In [8]:
def name_abbrev(name):
    # This function takes in a full name and changes it into first_intial.last_name
    names = name.split(' ')
    return '.'.join([names[0][0], names[1]])

In [9]:
pitchers['abbrev_name'] = pitchers['Name'].apply(name_abbrev)
pitchers.rename(columns = {'HardHit%':'HardHit%_P'}, inplace = True)
pitchers.drop_duplicates(subset = 'Name', inplace = True)
pitchers.head()

,Name,ERA,K/BB,HR/9,WHIP,xFIP,HardHit%_P,abbrev_name
192,Carlos Rodon,2.84,4.49,0.59,1.04,2.95,0.399,C.Rodon
291,Aaron Nola,3.38,8.08,0.87,0.98,2.84,0.319,A.Nola
300,Kevin Gausman,3.45,7.44,0.79,1.24,2.79,0.387,K.Gausman
88,Justin Verlander,1.78,6.04,0.69,0.83,3.27,0.349,J.Verlander
129,Sandy Alcantara,2.37,3.84,0.63,1.01,3.37,0.396,S.Alcantara


In [10]:
pitchers.to_csv('../data/pitching_stats.csv', index = False)

In [11]:
def remove_parenth(name):
    # This function uses regex to remove the parentheses and everything inside them from a string.
    import re
    return re.sub('([\(]).*?([\)])', '', name)

In [12]:
df['abbrev_name'] = df['OppStart'].apply(remove_parenth)  # Applying the above function
df.head()

,Date,Home,Opp,R,H,HR,BA,OPS,OppStart,Team,abbrev_name
0,Apr 7,False,SDP,4,3,1,0.115,0.554,Y.Darvish(71),ARI,Y.Darvish
1,Apr 8,False,SDP,0,2,0,0.091,0.364,S.Manaea(83),ARI,S.Manaea
2,Apr 9,False,SDP,2,5,1,0.116,0.426,J.Musgrove(62),ARI,J.Musgrove
3,Apr 10,False,SDP,5,5,2,0.129,0.516,N.Crismatt(58),ARI,N.Crismatt
4,Apr 12,False,HOU,1,4,1,0.130,0.518,L.Garcia(57),ARI,L.Garcia


In [13]:
batting = pb.team_batting(2022)  # Import team batting stats
batting.head()

,teamIDfg,Season,Team,Age,G,AB,PA,H,1B,2B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,22,2022,LAD,29,2075,4967,5618,1290,760,304,...,0.094,112.5,1583,0.416,3801,0.160,0.268,NaN,NaN,NaN
1,16,2022,ATL,29,2027,4968,5483,1257,756,271,...,0.109,116.8,1594,0.436,3654,0.146,0.276,NaN,NaN,NaN
2,14,2022,TOR,29,2210,5027,5571,1311,841,284,...,0.085,118.4,1747,0.443,3948,0.161,0.267,NaN,NaN,NaN
3,28,2022,STL,28,2140,5018,5632,1277,812,261,...,0.075,114.4,1471,0.372,3952,0.168,0.272,NaN,NaN,NaN
4,9,2022,NYY,29,2117,4908,5580,1178,737,202,...,0.101,119.8,1533,0.412,3719,0.177,0.285,NaN,NaN,NaN


In [14]:
batters = batting[['Team', 'wOBA', 'wRC+', 'OBP+', 'Barrel%', 'HardHit%']].copy()   # Only take the columns we plan to use
batters.head()

,Team,wOBA,wRC+,OBP+,Barrel%,HardHit%
0,LAD,0.341,122,107,0.094,0.416
1,ATL,0.330,110,101,0.109,0.436
2,TOR,0.328,115,105,0.085,0.443
3,STL,0.328,115,104,0.075,0.372
4,NYY,0.326,115,104,0.101,0.412


In [15]:
df_1 = df.merge(pitchers, how = 'left', left_on = 'abbrev_name', right_on = 'abbrev_name')     # Merge game log and pitching stats
df_1.dropna(inplace = True)
df_1.head()

,Date,Home,Opp,R,H,HR,BA,OPS,OppStart,Team,abbrev_name,Name,ERA,K/BB,HR/9,WHIP,xFIP,HardHit%_P
0,Apr 7,False,SDP,4,3,1,0.115,0.554,Y.Darvish(71),ARI,Y.Darvish,Yu Darvish,3.05,5.23,0.99,0.93,3.57,0.371
1,Apr 8,False,SDP,0,2,0,0.091,0.364,S.Manaea(83),ARI,S.Manaea,Sean Manaea,5.18,3.15,1.75,1.33,3.94,0.419
2,Apr 9,False,SDP,2,5,1,0.116,0.426,J.Musgrove(62),ARI,J.Musgrove,Joe Musgrove,3.16,4.85,1.15,1.07,3.43,0.325
3,Apr 10,False,SDP,5,5,2,0.129,0.516,N.Crismatt(58),ARI,N.Crismatt,Nabil Crismatt,2.98,2.82,0.57,1.20,3.51,0.363
4,Apr 12,False,HOU,1,4,1,0.130,0.518,L.Garcia(57),ARI,L.Garcia,Luis Garcia,4.04,3.40,1.41,1.17,3.81,0.349


In [16]:
df_2 = df_1.merge(batters, how = 'left', left_on = 'Team', right_on = 'Team')     # Merge game log and team batting stats
df_2.head()

,Date,Home,Opp,R,H,HR,BA,OPS,OppStart,Team,...,K/BB,HR/9,WHIP,xFIP,HardHit%_P,wOBA,wRC+,OBP+,Barrel%,HardHit%
0,Apr 7,False,SDP,4,3,1,0.115,0.554,Y.Darvish(71),ARI,...,5.23,0.99,0.93,3.57,0.371,0.305,93,97,0.068,0.358
1,Apr 8,False,SDP,0,2,0,0.091,0.364,S.Manaea(83),ARI,...,3.15,1.75,1.33,3.94,0.419,0.305,93,97,0.068,0.358
2,Apr 9,False,SDP,2,5,1,0.116,0.426,J.Musgrove(62),ARI,...,4.85,1.15,1.07,3.43,0.325,0.305,93,97,0.068,0.358
3,Apr 10,False,SDP,5,5,2,0.129,0.516,N.Crismatt(58),ARI,...,2.82,0.57,1.20,3.51,0.363,0.305,93,97,0.068,0.358
4,Apr 12,False,HOU,1,4,1,0.130,0.518,L.Garcia(57),ARI,...,3.40,1.41,1.17,3.81,0.349,0.305,93,97,0.068,0.358


In [17]:
df_2.drop(columns = ['OppStart', 'Name'], inplace = True) # Drop unnecessary columns
df_2.head()

,Date,Home,Opp,R,H,HR,BA,OPS,Team,abbrev_name,...,K/BB,HR/9,WHIP,xFIP,HardHit%_P,wOBA,wRC+,OBP+,Barrel%,HardHit%
0,Apr 7,False,SDP,4,3,1,0.115,0.554,ARI,Y.Darvish,...,5.23,0.99,0.93,3.57,0.371,0.305,93,97,0.068,0.358
1,Apr 8,False,SDP,0,2,0,0.091,0.364,ARI,S.Manaea,...,3.15,1.75,1.33,3.94,0.419,0.305,93,97,0.068,0.358
2,Apr 9,False,SDP,2,5,1,0.116,0.426,ARI,J.Musgrove,...,4.85,1.15,1.07,3.43,0.325,0.305,93,97,0.068,0.358
3,Apr 10,False,SDP,5,5,2,0.129,0.516,ARI,N.Crismatt,...,2.82,0.57,1.20,3.51,0.363,0.305,93,97,0.068,0.358
4,Apr 12,False,HOU,1,4,1,0.130,0.518,ARI,L.Garcia,...,3.40,1.41,1.17,3.81,0.349,0.305,93,97,0.068,0.358


In [18]:
df_2.to_csv('../data/game_logs.csv', index = False)    # Export to csv to be used for modeling

# Creating a csv that contains team batting stats for the season to be used in prediction modeling
---

In [19]:
games_played = dict(df['Team'].value_counts())   # Create a dictionary of games played by team
games_played = pd.DataFrame(list(games_played.items()))  # Turn it into a dataframe so it can be merged with the batting dataframe
games_played.columns = ['team', 'gp']
games_played.head()

,team,gp
0,STL,148
1,NYM,148
2,PIT,147
3,TOR,147
4,OAK,147


In [20]:
team_batting = batting[['Team', 'wOBA', 'wRC+', 'OBP+', 'Barrel%', 'HardHit%', 'H', 'HR', 'OPS', 'AVG']].copy()   # Take only columns we need
team_batting.rename(columns = {'AVG':'BA'}, inplace = True)
team_batting_df = team_batting.merge(games_played, left_on = 'Team', right_on = 'team') # Merge with previously created games_played df
team_batting_df['H'] = team_batting_df['H'] / team_batting_df['gp']      # Find H and HR/game
team_batting_df['HR'] = team_batting_df['HR'] / team_batting_df['gp']
team_batting_df.drop(columns = ['team', 'gp'], inplace = True)
team_batting_df.head()

,Team,wOBA,wRC+,OBP+,Barrel%,HardHit%,H,HR,OPS,BA
0,LAD,0.341,122,107,0.094,0.416,8.896552,1.358621,0.788,0.260
1,ATL,0.330,110,101,0.109,0.436,8.609589,1.506849,0.762,0.253
2,TOR,0.328,115,105,0.085,0.443,8.918367,1.204082,0.753,0.261
3,STL,0.328,115,104,0.075,0.372,8.628378,1.236486,0.752,0.254
4,NYY,0.326,115,104,0.101,0.412,8.068493,1.582192,0.749,0.240


In [21]:
team_batting_df.to_csv('../data/team_batting_stats.csv', index = False)  # Export to csv for use in prediction modeling